# Movielens 영화 추천
- 별점을 시청횟수로 가정 
- 3점 미만은 선호하지 않는다고 가정

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'counts', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.shape

(1000209, 4)

In [2]:
len(ratings.counts)
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['counts']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 countS로 바꿉니다.
# ratings.rename(columns={'rating':'count'}, inplace=True)
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies['title'] = movies['title'].str.lower()

In [6]:
movies['title']

0                         toy story (1995)
1                           jumanji (1995)
2                  grumpier old men (1995)
3                 waiting to exhale (1995)
4       father of the bride part ii (1995)
                       ...                
3878               meet the parents (2000)
3879            requiem for a dream (2000)
3880                      tigerland (2000)
3881               two family house (2000)
3882                 contender, the (2000)
Name: title, Length: 3883, dtype: object

In [73]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
artist_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

# 2.분석하기
- ratings에 있는 유니크한 영화 갯수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기수)

In [48]:
print(ratings['movie_id'].nunique())
print(ratings['user_id'].nunique())
movies_count = ratings.groupby('movie_id')['user_id'].count()
# movies_count.sort_values(ascending=False).head(30)
a = movies_count.sort_values(ascending=False).head(30)
a = list(a.keys())


3628
6039


# 3.선호하는 영화 5가지 골라 추가하기 

In [5]:
my_lovemv = ['toy story','jumanji','grumpier old men']
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 


# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['my']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

836478
836478
4


,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 4.CSR matrix 직접만들기

In [52]:
print(len(ratings.counts))
print(len(ratings.user_id))
print(len(ratings.movie_id))
print(num_artist)
print(num_user*num_artist)

836478
836478
836478
3628
21909492


In [57]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_artist = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)))


In [67]:
print('')

# 5. als_model = AlternatinngLeastSquares 모델 구성하여 훈련

In [68]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [69]:
als_model = AlternatingLeastSquares(factors=128, regularization=0.01, iterations=15, dtype=np.float32)

In [70]:
csr_data_transpose = csr_data.T
csr_data_transposecsr_data_transpose = csr_data.T


In [71]:
als_model.fit(csr_data_transpose)

100%|██████████| 15/15 [00:02<00:00,  5.83it/s]


In [103]:
zimin, black_eyed_peas = user_to_idx[100], artist_to_idx[141]
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]


In [104]:
zimin_vector
black_eyed_peas_vector
np.dot(zimin_vector, black_eyed_peas_vector)

0.0051948195

# 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화를 하나를 골라 훈련된 모델이 예측한 선호도 파악

# 7. 내가 좋아하는 영황와 비슷한 영화를 추천받기

# 8. 내가 가장 좋아할 만한 영화들 추천받기